In [10]:
MSMSCMD = "E:\\Tesis\\SLIC_2_Atoms\\Parametrized\\MSMS\\msms.exe"
basedir = "E:\\Tesis\\SLIC_2_Atoms_NEW"
MESHPATH = "E:\\Tesis\\SLIC_2_Atoms_NEW\\MESH"
head = {"distance": ["dielectric_interfaces",
                     "stern_layers",
                     "PB_solvation_energy++",
                     "PB_PMF++",
                     "SLIC_solvation_energy++",
                     "SLIC_PMF++",
                     "SLIC_proposed_solvation_energy++",
                     "SLIC_proposed_PMF++",
                     "PB_solvation_energy+-",
                     "PB_PMF+-",
                     "SLIC_solvation_energy+-",
                     "SLIC_PMF+-",
                     "SLIC_proposed_solvation_energy+-",
                     "SLIC_proposed_PMF+-",
                     "SLIC_solvation_energy--",
                     "SLIC_PMF--",
                     "SLIC_proposed_solvation_energy--",
                     "SLIC_proposed_PMF--",
                     "dielectric_mesh",
                     "stern_mesh"]}
info = {"ion": ["shared","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "3.0": ["shared","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "3.5": ["shared","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "4.0": ["shared","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "03.0": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "03.5": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "04.0": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "4.5": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "5.0": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "5.5": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "6.0": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "6.5": ["separate","shared",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "06.5": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "7.0": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "7.5": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "8.0": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "8.5": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "9.0": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "9.5": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]],
        "10.0": ["separate","separate",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[],[]]
       }

STERNWIDTH = 2
ITER = 20
TOL = 1e-8
MAXITER_GM = 1000
piter = True
pvalues = True
pstatus = True

In [11]:
alpha = 0.94
beta = -18.72
gamma = -0.47

#0,94	-18,72	-0,47

In [12]:
# Se importan las lirerías necesarias:
import os
import shutil
import bempp.api
import numpy as np
from scipy.sparse.linalg import gmres
import time

In [13]:
os.chdir(basedir)

In [ ]:
os.chdir(basedir)
os.chdir("MESH")

for key in info.keys():
    try:
        os.mkdir(key)
    except:
        pass
    #Para ion
    if key == "ion":
        #Crear directorio
        
        #Añadir linea a base.xyzr para interfaz dielectrica
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                f1.write("\n0.0\t0.0\t0.0\t1.27\n")
        
        #Correr NanoShaper
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\dielectric '
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 15.0'
                  +' -no_header'
                 )
        
        #Añadir linea a xyzr para stern        
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                f1.write("\n0.0\t0.0\t0.0\t"+str(STERNWIDTH+1.27)+"\n")
        
        #Correr NanoShaper
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\stern '
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 3.0'
                  +' -no_header'
                 )
        
    #Para caso de interfaz y stern layers compartidos:    
    elif  info[key][0] == "shared" and info[key][1] == "shared":
        #Crear directorio
        #os.mkdir(key)
        
        #Añadir linea a xyzr para interfaz dielectrica
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                f1.write("\n0.0\t0.0\t0.0\t1.27\n")
                f1.write(key+"\t0.0\t0.0\t1.27\n")
        
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\dielectric '
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 15.0'
                  +' -no_header'
                 )
        
        #Añadir linea a xyzr para stern layer
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                f1.write("\n0.0\t0.0\t0.0\t"+str(STERNWIDTH+1.27)+"\n")
                f1.write(key+"\t0.0\t0.0\t"+str(STERNWIDTH+1.27)+"\n")
        
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\stern'
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 3.0'
                  +' -no_header'
                 )
        
    #Para caso de interfaz separada y stern layers compartidas:    
    elif  info[key][0] == "separate" and info[key][1] == "shared":
        #Crear directorio
        #os.mkdir(key)
        
        #Copiar malla de ion para primera interfaz dielectrica:
        shutil.copy(MESHPATH+"\\ion\\dielectric.face",
                    MESHPATH+"\\"+key+"\\dielectric_1.face")
        shutil.copy(MESHPATH+"\\ion\\dielectric.vert",
                    MESHPATH+"\\"+key+"\\dielectric_1.vert")
        
        #Añadir linea a xyzr para interfaz dielectrica 2
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                    #print(line)
                f1.write("\n"+key+"\t0.0\t0.0\t1.27\n")
                #print("\n"+key+"\t0.0\t0.0\t1.27\n")
        
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\dielectric_2'
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 15.0'
                  +' -no_header'
                 )
        
        
        #Añadir linea a xyzr para stern layer
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                f1.write("\n0.0\t0.0\t0.0\t"+str(STERNWIDTH+1.27)+"\n")
                f1.write("\n"+key+"\t0.0\t0.0\t"+str(STERNWIDTH+1.27)+"\n")
        
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\stern'
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 3.0'
                  +' -no_header'
                 )
        
    #Para caso de interfaz separada y stern layers separads:    
    elif  info[key][0] == "separate" and info[key][1] == "separate":
        #Crear directorio
        #os.mkdir(key)
        
        #Copiar malla de ion para primera interfaz dielectrica:
        shutil.copy(MESHPATH+"\\ion\\dielectric.face",
                    MESHPATH+"\\"+key+"\\dielectric_1.face")
        shutil.copy(MESHPATH+"\\ion\\dielectric.vert",
                    MESHPATH+"\\"+key+"\\dielectric_1.vert")
        
        #Añadir linea a xyzr para interfaz dielectrica 2
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                f1.write("\n"+key+"\t0.0\t0.0\t1.27\n")
        
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\dielectric_2'
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 15.0'
                  +' -no_header'
                 )
        
        #Copiar malla de ion para primera stern layer:
        shutil.copy(MESHPATH+"\\ion\\stern.face",
                    MESHPATH+"\\"+key+"\\stern_1.face")
        shutil.copy(MESHPATH+"\\ion\\stern.vert",
                    MESHPATH+"\\"+key+"\\stern_1.vert")
        
        #Añadir linea a xyzr para stern        
        with open(MESHPATH+"\\base.xyzr") as f:
            with open(MESHPATH+"\\"+key+"\\mesh.xyzr", "w") as f1:
                for line in f:
                    f1.write(line)
                f1.write("\n"+key+"\t0.0\t0.0\t"+str(STERNWIDTH+1.27)+"\n")
        
        os.system(MSMSCMD
                  +' -if '
                  +MESHPATH+'\\'+key+'\\mesh.xyzr'
                  +' -of '
                  +MESHPATH+'\\'+key+'\\stern_2'
                  +' -p'
                  +' 1.4'
                  +' -d'
                  +' 3.0'
                  +' -no_header'
                 )
        
os.chdir(basedir)

In [14]:
# Función para crear mallas a partir de archivos .face y .vert
def import_msms_mesh(mesh_face_path, mesh_vert_path):
    face = open(mesh_face_path,'r').read()
    vert = open(mesh_vert_path,'r').read()

    faces = np.vstack(np.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1 
    verts = np.vstack(np.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    #grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    grid = bempp.api.Grid(verts.transpose(), faces.transpose())
    return grid

# Numba no es compatible con la función norm de numpy, por lo que debemos crearla dentro de Numba:
from numba import njit
@njit
def impl1(F):
    for i in range(F.shape[0]):
        nrm = np.linalg.norm(F[i,:3])
        F[i] = nrm
    return F[:,0]

def pristatus(b, key):
    if b:
        print("Solving for "+key)
    return None

def priiter(b,i,en):
    if b:
        print("Iteration: "+str(i)+", Energía: "+str(en))
    return None
        
def privalues(b,val,key,at,method):
    if b:
        print("Solvation Energy, "+key+", "+method+": "+str(val))
        print(at)

In [15]:
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import matplotlib as mpl
from matplotlib import pyplot as plt
def plot_mesh_to_file(directory,grid): #USAR PATH COMPLETO DE ARCHIVO
    vertices = grid.vertices
    elements = grid.elements
    fig = ff.create_trisurf(
                x=vertices[0, :],
                y=vertices[1, :],
                z=vertices[2, :],
                simplices=elements.T,
                color_func=elements.shape[1] * ["rgb(255, 222, 173)"],
            )
    fig["layout"]["scene"].update(go.layout.Scene(aspectmode="data"))
    #plotly.offline.iplot(fig)
    fig.write_html(directory+".html")
    fig.write_image(directory+".png",scale =4.0)
    
def plot_solution_to_file(directory,solution,atoms): #USAR PATH COMPLETO
    grid = solution.space.grid
    vertices = grid.vertices
    elements = grid.elements
    local_coordinates = np.array([[1.0 / 3], [1.0 / 3]])
    intensity_1 = np.zeros(grid.entity_count(0), dtype="float64")
    for element in grid.entity_iterator(0):
        index = element.index
        local_values = np.real((solution.evaluate(index, local_coordinates)))
        intensity_1[index] = local_values.flatten()

    i_0 = elements[0,:]
    j_0 = elements[1,:]
    k_0 = elements[2,:]

    triangles = np.vstack((i_0,j_0,k_0)).T

    x_0 = vertices[0, :]
    y_0 = vertices[1, :]
    z_0 = vertices[2, :]
    vertices = np.vstack((x_0,y_0,z_0)).T
    tri_points = vertices[triangles]

    #extract the lists of x, y, z coordinates of the triangle vertices and connect them by a line
    Xe = []
    Ye = []
    Ze = []
    for T in tri_points:
        Xe.extend([T[k%3][0] for k in range(4)]+[ None])
        Ye.extend([T[k%3][1] for k in range(4)]+[ None])
        Ze.extend([T[k%3][2] for k in range(4)]+[ None])

    #define the trace for triangle sides
    lines = go.Scatter3d(
                       x=Xe,
                       y=Ye,
                       z=Ze,
                       mode='lines',
                       name='',
                       line=dict(color= 'rgb(0,0,0)', width=1))  

    fig = go.Figure(data=[
        go.Mesh3d(
            # 8 vertices of a cube
            x=x_0,
            y=y_0,
            z=z_0,
            colorscale="Jet",
            # Intensity of each vertex, which will be interpolated and color-coded
            intensity = intensity_1,
            intensitymode='cell',
            # i, j and k give the vertices of triangles
            i = i_0,
            j = j_0,
            k = k_0,
            name='y',
        ),lines
    ])
    fig["layout"]["scene"].update(go.layout.Scene(aspectmode="data"))
    fig.write_html(directory+'_'+str(atoms[0][3])+'_'+str(atoms[1][3])+".html")
    fig.write_image(directory+'_'+str(atoms[0][3])+'_'+str(atoms[1][3])+".png",scale =4.0)

In [46]:
def ion():
    en_sol = 0
    energiesonly.write('\nion,')
    phidphionly.write('\nion')
    #Importan mallas de ion 
    pristatus(pstatus,key)
    grid_dielectric = import_msms_mesh(MESHPATH+"\\"+key+"\\dielectric.face",
                                       MESHPATH+"\\"+key+"\\dielectric.vert")

    grid_stern = import_msms_mesh(MESHPATH+"\\"+key+"\\stern.face",
                                  MESHPATH+"\\"+key+"\\stern.vert")

    #Guarda malla en html y png
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\dielectric",grid_dielectric)
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\stern",grid_stern)
    

    # Se crean los espacios funcionales:
    #Protein
    dirichlet_ion = bempp.api.function_space(grid_dielectric, "DP", 0)
    neumann_ion = bempp.api.function_space(grid_dielectric, "DP", 0)

    #Stern
    dirichlet_stern = bempp.api.function_space(grid_stern, "DP", 0)
    neumann_stern = bempp.api.function_space(grid_stern, "DP", 0)

    om = 0 #Debye lenght
    ep_i = 1
    ep_ii = 80
    ep_iii = 80

    #Operators:
    identity_ion = bempp.api.operators.boundary.sparse.identity(
        dirichlet_ion, dirichlet_ion, neumann_ion)

    K_L_I_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion, dirichlet_ion, neumann_ion)
    
    K_L_I_a_a_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
        dirichlet_ion, dirichlet_ion, neumann_ion)

    V_L_I_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion, dirichlet_ion, neumann_ion)

    K_L_II_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion, dirichlet_ion, neumann_ion)

    V_L_II_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion, dirichlet_ion, neumann_ion)

    K_L_II_a_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern, dirichlet_ion, neumann_ion) #OJO ACA

    V_L_II_a_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern, dirichlet_ion, neumann_ion) #OJO ACA

    K_L_II_b_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion, dirichlet_stern, neumann_stern) #OJO ACA

    V_L_II_b_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion, dirichlet_stern, neumann_stern) #OJO ACA

    identity_stern = bempp.api.operators.boundary.sparse.identity(
        dirichlet_stern, dirichlet_stern, neumann_stern)

    K_L_II_b_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern, dirichlet_stern, neumann_stern)

    V_L_II_b_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern, dirichlet_stern, neumann_stern)

    K_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.double_layer(
        dirichlet_stern, dirichlet_stern, neumann_stern, om)

    V_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.single_layer(
        neumann_stern, dirichlet_stern, neumann_stern, om)

    for q, index in [(1,6),(-1,16)]:
        atoms = np.array([[0,0,0,q]])
        qtotal=np.sum(atoms[:,3])

        
        @bempp.api.real_callable
        def rhs_function(x, n, domain_index, result):
            result[:] = np.sum(atoms[:,3]/(4 * np.pi * ep_i * impl1(x-atoms[:,:3])))
        # Se crea una GridFunction conteniendo el lado derecho de la equación:
        rhs_1 = bempp.api.GridFunction(dirichlet_ion, fun=rhs_function)
        rhs_2 = np.zeros(neumann_ion.global_dof_count)
        rhs_3 = np.zeros(dirichlet_stern.global_dof_count)
        rhs_4 = np.zeros(neumann_stern.global_dof_count)
        rhs = np.concatenate([rhs_1.coefficients,rhs_2,rhs_3,rhs_4])
        
        @bempp.api.real_callable
        def en_function(x, n, domain_index, result):
            result[:] = np.sum(np.dot(n,(np.transpose(x-atoms[:,:3])/
                                         (4 * np.pi * impl1(x-atoms[:,:3])**3))*
                                      (atoms[:,3])))

        en_1 = bempp.api.GridFunction(dirichlet_ion, fun=en_function)
        #ACA EL SIGNO MENOS QUE CAMBIÉ

        def f(sigma,fun):
            Ksigma = K_L_I_a_a_adjoint * sigma
            En = fun - Ksigma
            mu = - alpha * np.tanh(- gamma)
            h = alpha * np.tanh(beta * En.coefficients - gamma) + mu
            f = ep_i/(ep_ii - ep_i) - h
            f_div = f/(1+f)
            return f_div 


        @bempp.api.real_callable
        def d1_function(x, n, domain_index, result):
            result[:] = np.sum(atoms[:,3]/(impl1(x-atoms[:,:3])))

        d1_fun = bempp.api.GridFunction(neumann_stern, fun = d1_function) 
        d1_mat = -(qtotal/ep_ii) * d1_fun.coefficients/np.mean(d1_fun.coefficients)
        d1_gridfun = bempp.api.GridFunction(neumann_stern, coefficients = d1_mat)
        d1_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(d1_gridfun, 
                                                    neumann_stern, neumann_stern, neumann_stern)


        d2 = 1

        sigma = bempp.api.GridFunction(dirichlet_ion, coefficients = np.zeros(
            dirichlet_ion.global_dof_count))

        for i in range(ITER): 
            blocked = bempp.api.BlockedOperator(4, 4)
            if i != 0:
                d2 = solution_neumann_stern.integrate()[0] 
                f_it = f(sigma,en_1)
                f_fun = bempp.api.GridFunction(neumann_ion, coefficients = f_it)
                f_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun, 
                                        neumann_ion, neumann_ion, neumann_ion)
                blocked[1,1] =  V_L_II_a_a * f_op
                blocked[2,1] =  V_L_II_b_a * f_op
                blocked[3,3] =  (1/d2) * V_Y_III_b_b * d1_op 
            else:
                blocked[1,1] =  V_L_II_a_a
                blocked[2,1] =  V_L_II_b_a
                blocked[3,3] =  (1/d2) * V_Y_III_b_b 

            blocked[0,0] = 0.5 * identity_ion + K_L_I_a_a 
            blocked[0,1] = - V_L_I_a_a

            blocked[1,0] = 0.5 * identity_ion - K_L_II_a_a
            blocked[1,2] = K_L_II_a_b 
            blocked[1,3] = - V_L_II_a_b

            blocked[2,0] = - K_L_II_b_a
            blocked[2,2] = 0.5 * identity_stern + K_L_II_b_b
            blocked[2,3] = - V_L_II_b_b

            blocked[3,2] = 0.5 * identity_stern - K_Y_III_b_b


            blocked = blocked.strong_form()

            priiter(piter,i,en_sol)

            x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol = TOL)

            solution_neumann_stern = bempp.api.GridFunction(neumann_stern, coefficients = 
                                                            x[(dirichlet_ion.global_dof_count+
                                                             neumann_ion.global_dof_count+
                                                              dirichlet_stern.global_dof_count):])
            #Sigma Ec. 46
            solution_neumann_ion = bempp.api.GridFunction(neumann_ion, coefficients = 
                                                            x[dirichlet_ion.global_dof_count:
                                                             (dirichlet_ion.global_dof_count+
                                                              neumann_ion.global_dof_count)])

            solution_dirichlet_ion = bempp.api.GridFunction(dirichlet_ion, coefficients = 
                                                            x[:dirichlet_ion.global_dof_count])

            solution_dirichlet_stern = bempp.api.GridFunction(dirichlet_stern, coefficients = 
                                                x[(dirichlet_ion.global_dof_count+
                                                 neumann_ion.global_dof_count):
                                                  (dirichlet_ion.global_dof_count+
                                                 neumann_ion.global_dof_count+
                                                  dirichlet_stern.global_dof_count)])

            rhs_sigma = V_L_I_a_a * solution_neumann_ion - (- 0.5 * identity_ion + K_L_I_a_a) * solution_dirichlet_ion

            sigma, info_2 = bempp.api.linalg.gmres(V_L_I_a_a, rhs_sigma, 
                                                   tol=TOL, maxiter = MAXITER_GM) 

            S0_q = bempp.api.operators.potential.laplace.single_layer(neumann_ion, atoms[:,:3].transpose())
            D0_q = bempp.api.operators.potential.laplace.double_layer(dirichlet_ion, atoms[:,:3].transpose())
            phi_q = S0_q * solution_neumann_ion - D0_q * solution_dirichlet_ion
            en_sol = 2 * np.pi * 332.064 * np.sum(atoms[:,3] * phi_q).real


        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_dielectric_SLIC_PROP",solution_dirichlet_ion,np.concatenate((atoms,atoms)))
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_dielectric_SLIC_PROP",solution_neumann_ion,np.concatenate((atoms,atoms)))
        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_stern_SLIC_PROP",solution_dirichlet_stern,np.concatenate((atoms,atoms)))
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_stern_SLIC_PROP",solution_neumann_stern,np.concatenate((atoms,atoms)))
        
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_dielectric_'+str(atoms[0][3])+'.csv', solution_dirichlet_ion.coefficients, delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_dielectric_'+str(atoms[0][3])+'.csv', solution_neumann_ion.coefficients, delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_stern_'+str(atoms[0][3])+'.csv', solution_dirichlet_stern.coefficients, delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_stern_'+str(atoms[0][3])+'.csv', solution_neumann_stern.coefficients, delimiter=",")
        
        info[key][index] = en_sol
        privalues(pvalues,en_sol,key,atoms,"SLIC Proposed")
        energiesonly.write(str(en_sol)+',')
        phidphionly.write(',SLICPROP_phidielectric,'+str(np.mean(solution_dirichlet_ion.coefficients))+
                      ',SLICPROP_dphidielectric,'+str(np.mean(solution_neumann_ion.coefficients))+
                     ',SLICPROP_phistern,'+str(np.mean(solution_dirichlet_stern.coefficients))+
                     ',SLICPROP_dphistern'+str(np.mean(solution_neumann_stern.coefficients)))
    

In [47]:
def shared_shared():
    en_sol = 0
    energiesonly.write('\n'+str(key)+',')
    phidphionly.write('\n'+str(key)+',')
    pristatus(pstatus,key)
    #Importan mallas de ion 
    grid_dielectric = import_msms_mesh(MESHPATH+"\\"+key+"\\dielectric.face",
                                       MESHPATH+"\\"+key+"\\dielectric.vert")

    grid_stern = import_msms_mesh(MESHPATH+"\\"+key+"\\stern.face",
                                  MESHPATH+"\\"+key+"\\stern.vert")

    #Se guarda malla en lista
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\dielectric",grid_dielectric)
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\stern",grid_stern)

    # Se crean los espacios funcionales:
    #Protein
    dirichlet_ion = bempp.api.function_space(grid_dielectric, "DP", 0)
    neumann_ion = bempp.api.function_space(grid_dielectric, "DP", 0)

    #Stern
    dirichlet_stern = bempp.api.function_space(grid_stern, "DP", 0)
    neumann_stern = bempp.api.function_space(grid_stern, "DP", 0)

    om = 0 #Debye lenght
    ep_i = 1
    ep_ii = 80
    ep_iii = 80

    #Operators:
    identity_ion = bempp.api.operators.boundary.sparse.identity(
        dirichlet_ion, dirichlet_ion, neumann_ion)

    K_L_I_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion, dirichlet_ion, neumann_ion)
    
    K_L_I_a_a_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
        dirichlet_ion, dirichlet_ion, neumann_ion)

    V_L_I_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion, dirichlet_ion, neumann_ion)

    K_L_II_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion, dirichlet_ion, neumann_ion)

    V_L_II_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion, dirichlet_ion, neumann_ion)

    K_L_II_a_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern, dirichlet_ion, neumann_ion) #OJO ACA

    V_L_II_a_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern, dirichlet_ion, neumann_ion) #OJO ACA

    K_L_II_b_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion, dirichlet_stern, neumann_stern) #OJO ACA

    V_L_II_b_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion, dirichlet_stern, neumann_stern) #OJO ACA

    identity_stern = bempp.api.operators.boundary.sparse.identity(
        dirichlet_stern, dirichlet_stern, neumann_stern)

    K_L_II_b_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern, dirichlet_stern, neumann_stern)

    V_L_II_b_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern, dirichlet_stern, neumann_stern)

    K_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.double_layer(
        dirichlet_stern, dirichlet_stern, neumann_stern, om)

    V_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.single_layer(
        neumann_stern, dirichlet_stern, neumann_stern, om)

    for q1,q2,index in [(1,1,6),(1,-1,12),(-1,-1,16)]:
        atoms = np.array([[0,0,0,q1],[float(key),0,0,q2]])
        qtotal=np.sum(atoms[:,3]) 

        @bempp.api.real_callable
        def rhs_function(x, n, domain_index, result):
            
            result[:] = np.sum(atoms[:,3]/(4 * np.pi * ep_i * impl1(x-atoms[:,:3])))
        # Se crea una GridFunction conteniendo el lado derecho de la equación:
        rhs_1 = bempp.api.GridFunction(dirichlet_ion, fun=rhs_function)
        rhs_2 = np.zeros(neumann_ion.global_dof_count)
        rhs_3 = np.zeros(dirichlet_stern.global_dof_count)
        rhs_4 = np.zeros(neumann_stern.global_dof_count)
        rhs = np.concatenate([rhs_1.coefficients,rhs_2,rhs_3,rhs_4])
        
        @bempp.api.real_callable
        def en_function(x, n, domain_index, result):
            
            result[:] = np.sum(np.dot(n,(np.transpose(x-atoms[:,:3])/
                                         (4 * np.pi * impl1(x-atoms[:,:3])**3))*
                                      (atoms[:,3])))

        en_1 = bempp.api.GridFunction(dirichlet_ion, fun=en_function)
        #ACA EL SIGNO MENOS QUE CAMBIÉ

        def f(sigma,fun):
            Ksigma = K_L_I_a_a_adjoint * sigma
            En = fun - Ksigma
            mu = - alpha * np.tanh(- gamma)
            h = alpha * np.tanh(beta * En.coefficients - gamma) + mu
            f = ep_i/(ep_ii - ep_i) - h
            f_div = f/(1+f)
            return f_div 

        d2 = 1

        @bempp.api.real_callable
        def d1_function(x, n, domain_index, result):
            result[:] = np.sum(atoms[:,3]/(impl1(x-atoms[:,:3])))

        d1_fun = bempp.api.GridFunction(neumann_stern, fun = d1_function) 
        if qtotal == 0:
            d1_mat = -(1/ep_ii) * d1_fun.coefficients
            #d1_mat = -(1/ep_ii) * d1_fun.coefficients/np.mean(d1_fun.coefficients)
            #d1_mat =   d1_mat- np.mean(d1_mat)
            #Se escala y se centra en cero
        else:
            d1_mat = -(qtotal/ep_ii) * d1_fun.coefficients/np.mean(d1_fun.coefficients)
        d1_gridfun = bempp.api.GridFunction(neumann_stern, coefficients = d1_mat)
        d1_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(d1_gridfun, 
                                                    neumann_stern, neumann_stern, neumann_stern)

        for i in range(ITER): 
            blocked = bempp.api.BlockedOperator(4, 4)
            if i != 0:
                d2 = solution_neumann_stern.integrate()[0] 
                f_it = f(sigma,en_1)
                f_fun = bempp.api.GridFunction(neumann_ion, coefficients = f_it)
                f_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun, 
                                        neumann_ion, neumann_ion, neumann_ion)
                blocked[1,1] =  V_L_II_a_a * f_op
                blocked[2,1] =  V_L_II_b_a * f_op
                blocked[3,3] =  (1/d2) * V_Y_III_b_b * d1_op 
            else:
                blocked[1,1] =  V_L_II_a_a
                blocked[2,1] =  V_L_II_b_a
                blocked[3,3] =  (1/d2) * V_Y_III_b_b 


            blocked[0,0] = 0.5 * identity_ion + K_L_I_a_a 
            blocked[0,1] = - V_L_I_a_a

            blocked[1,0] = 0.5 * identity_ion - K_L_II_a_a
            blocked[1,2] = K_L_II_a_b 
            blocked[1,3] = - V_L_II_a_b

            blocked[2,0] = - K_L_II_b_a
            blocked[2,2] = 0.5 * identity_stern + K_L_II_b_b
            blocked[2,3] = - V_L_II_b_b

            blocked[3,2] = 0.5 * identity_stern - K_Y_III_b_b

            blocked = blocked.strong_form()

            priiter(piter,i,en_sol)

            x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol = TOL)

            solution_neumann_stern = bempp.api.GridFunction(neumann_stern, coefficients = 
                                                            x[(dirichlet_ion.global_dof_count+
                                                             neumann_ion.global_dof_count+
                                                              dirichlet_stern.global_dof_count):])
            #Sigma Ec. 46
            solution_neumann_ion = bempp.api.GridFunction(neumann_ion, coefficients = 
                                                            x[dirichlet_ion.global_dof_count:
                                                             (dirichlet_ion.global_dof_count+
                                                              neumann_ion.global_dof_count)])

            solution_dirichlet_ion = bempp.api.GridFunction(dirichlet_ion, coefficients = 
                                                            x[:dirichlet_ion.global_dof_count])

            solution_dirichlet_stern = bempp.api.GridFunction(dirichlet_stern, coefficients = 
                                                x[(dirichlet_ion.global_dof_count+
                                                 neumann_ion.global_dof_count):
                                                  (dirichlet_ion.global_dof_count+
                                                 neumann_ion.global_dof_count+
                                                  dirichlet_stern.global_dof_count)])

            rhs_sigma = V_L_I_a_a * solution_neumann_ion - (- 0.5 * identity_ion + K_L_I_a_a) * solution_dirichlet_ion

            sigma, info_2 = bempp.api.linalg.gmres(V_L_I_a_a, rhs_sigma, 
                                                   tol=TOL, maxiter = MAXITER_GM) 

            S0_q = bempp.api.operators.potential.laplace.single_layer(neumann_ion, atoms[:,:3].transpose())
            D0_q = bempp.api.operators.potential.laplace.double_layer(dirichlet_ion, atoms[:,:3].transpose())
            phi_q = S0_q * solution_neumann_ion - D0_q * solution_dirichlet_ion
            en_sol = 2 * np.pi * 332.064 * np.sum(atoms[:,3] * phi_q).real

        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_dielectric_SLIC_PROP",solution_dirichlet_ion,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_dielectric_SLIC_PROP",solution_neumann_ion,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_stern_SLIC_PROP",solution_dirichlet_stern,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_stern_SLIC_PROP",solution_neumann_stern,atoms)
        
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_dielectric_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', solution_dirichlet_ion.coefficients, delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_dielectric_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', solution_neumann_ion.coefficients, delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_stern_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', solution_dirichlet_stern.coefficients, delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_stern_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', solution_neumann_stern.coefficients, delimiter=",")
        
        
        info[key][index] = en_sol  
        privalues(pvalues,en_sol,key,atoms,"SLIC Proposed")
        energiesonly.write(str(en_sol)+',')
        
        phidphionly.write(',SLICPROP_phidielectric,'+str(np.mean(solution_dirichlet_ion.coefficients))+
                      ',SLICPROP_dphidielectric,'+str(np.mean(solution_neumann_ion.coefficients))+
                     ',SLICPROP_phistern,'+str(np.mean(solution_dirichlet_stern.coefficients))+
                     ',SLICPROP_dphistern'+str(np.mean(solution_neumann_stern.coefficients)))

In [48]:
def separate_shared():
    en_sol = 0
    energiesonly.write('\n'+str(key)+',')
    phidphionly.write('\n'+str(key)+',')
    pristatus(pstatus,key)
    #Importan mallas de ion 
    grid_dielectric_1 = import_msms_mesh(MESHPATH+"\\"+key+"\\dielectric_1.face",
                                         MESHPATH+"\\"+key+"\\dielectric_1.vert")
    grid_dielectric_2 = import_msms_mesh(MESHPATH+"\\"+key+"\\dielectric_2.face",
                                         MESHPATH+"\\"+key+"\\dielectric_2.vert")

    grid_stern = import_msms_mesh(MESHPATH+"\\"+key+"\\stern.face",
                                  MESHPATH+"\\"+key+"\\stern.vert")

    #Se guarda malla en lista
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\dielectric_1",grid_dielectric_1)
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\dielectric_2",grid_dielectric_2)
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\stern",grid_stern)

    # Se crean los espacios funcionales:
    #Ion 1
    dirichlet_ion_1 = bempp.api.function_space(grid_dielectric_1, "DP", 0)
    neumann_ion_1 = bempp.api.function_space(grid_dielectric_1, "DP", 0)

    #Ion 2
    dirichlet_ion_2 = bempp.api.function_space(grid_dielectric_2, "DP", 0)
    neumann_ion_2 = bempp.api.function_space(grid_dielectric_2, "DP", 0)

    #Stern
    dirichlet_stern = bempp.api.function_space(grid_stern, "DP", 0)
    neumann_stern = bempp.api.function_space(grid_stern, "DP", 0)

    om = 0 #Debye lenght
    ep_iv = 80
    ep_iii = 80
    ep_ii = 1
    ep_i = 1

    #Operators:
    # Fila 1:
    identity_ion_1 = bempp.api.operators.boundary.sparse.identity(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)

    K_L_I_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)
    
    K_L_I_a_a_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)

    G_L_I_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_1, dirichlet_ion_1, neumann_ion_1)

    #Fila 2:
    K_L_III_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)

    G_L_III_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_1, dirichlet_ion_1, neumann_ion_1)

    K_L_III_a_c = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern, dirichlet_ion_1, neumann_ion_1)

    G_L_III_a_c = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern, dirichlet_ion_1, neumann_ion_1) 

    K_L_III_a_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_2, dirichlet_ion_1, neumann_ion_1) 

    G_L_III_a_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_2, dirichlet_ion_1, neumann_ion_1) 

    #Fila 3:
    identity_ion_2 = bempp.api.operators.boundary.sparse.identity(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)

    K_L_II_b_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)
    
    K_L_II_b_b_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)

    G_L_II_b_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_2, dirichlet_ion_2, neumann_ion_2)


    #FIla 4:
    K_L_III_b_c = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern, dirichlet_ion_2, neumann_ion_2)

    G_L_III_b_c = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern, dirichlet_ion_2, neumann_ion_2) 

    K_L_III_b_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)

    G_L_III_b_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_2, dirichlet_ion_2, neumann_ion_2)

    K_L_III_b_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_1, dirichlet_ion_2, neumann_ion_2) 

    G_L_III_b_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_1, dirichlet_ion_2, neumann_ion_2) 

    #Fila 5:
    K_L_III_c_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_1, dirichlet_stern, neumann_stern)

    G_L_III_c_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_1, dirichlet_stern, neumann_stern)

    identity_stern = bempp.api.operators.boundary.sparse.identity(
        dirichlet_stern, dirichlet_stern, neumann_stern)

    K_L_III_c_c = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern, dirichlet_stern, neumann_stern)

    G_L_III_c_c = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern, dirichlet_stern, neumann_stern)

    K_L_III_c_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_2, dirichlet_stern, neumann_stern)

    G_L_III_c_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_2, dirichlet_stern, neumann_stern)

    #Fila 6:
    K_Y_IV_c_c = bempp.api.operators.boundary.modified_helmholtz.double_layer(
        dirichlet_stern, dirichlet_stern, neumann_stern, om)

    G_Y_IV_c_c = bempp.api.operators.boundary.modified_helmholtz.single_layer(
        neumann_stern, dirichlet_stern, neumann_stern, om)

    #SLIC proposed method
    for q1,q2,index in [(1,1,6),(1,-1,12),(-1,-1,16)]:
        atoms = np.array([[0,0,0,q1],[float(key),0,0,q2]])
        qtotal=np.sum(atoms[:,3])
        atoms_1 = np.array([atoms[0]])
        atoms_2 = np.array([atoms[1]])
        
        # Se mapea la función analítica del lado derecho de la equación a la malla creada en Bempp:
        @bempp.api.real_callable
        def rhs_function_1(x, n, domain_index, result):
            
            result[:] = atoms_1[:,3]/(4 * np.pi * ep_i * impl1(x-atoms_1[:,:3]))

        @bempp.api.real_callable
        def rhs_function_2(x, n, domain_index, result):
            
            result[:] = atoms_2[:,3]/(4 * np.pi * ep_ii * impl1(x-atoms_2[:,:3]))

        # Se crea una GridFunction conteniendo el lado derecho de la equación:
        rhs_1 = bempp.api.GridFunction(dirichlet_ion_1, fun=rhs_function_1)
        rhs_2 = np.zeros(neumann_ion_1.global_dof_count)
        rhs_3 = bempp.api.GridFunction(dirichlet_ion_2, fun=rhs_function_2)
        rhs_4 = np.zeros(neumann_ion_2.global_dof_count)
        rhs_5 = np.zeros(dirichlet_stern.global_dof_count)
        rhs_6 = np.zeros(neumann_stern.global_dof_count)
        rhs = np.concatenate([rhs_1.coefficients,rhs_2,rhs_3.coefficients,rhs_4,rhs_5,rhs_6])
        
        @bempp.api.real_callable
        def en_function_1(x, n, domain_index, result):
            
            result[:] = np.sum(np.dot(n,(np.transpose(x-atoms_1[:,:3])/
                                         (4 * np.pi * impl1(x-atoms_1[:,:3])**3))*
                                      (atoms_1[:,3])))

        @bempp.api.real_callable
        def en_function_2(x, n, domain_index, result):
            
            result[:] = np.sum(np.dot(n,(np.transpose(x-atoms_2[:,:3])/
                                         (4 * np.pi * impl1(x-atoms_2[:,:3])**3))*
                                      (atoms_2[:,3])))

        en_1 = bempp.api.GridFunction(dirichlet_ion_1, fun=en_function_1)
        en_2 = bempp.api.GridFunction(dirichlet_ion_2, fun=en_function_2)
        
        def f(sigma,fun,K,ep_in,ep_out):
            Ksigma = K * sigma 
            En = fun - Ksigma
            mu = - alpha * np.tanh(- gamma)
            h = alpha * np.tanh(beta * En.coefficients - gamma) + mu
            f = ep_in/(ep_out - ep_in) - h
            f_div = f/(1+f)
            return f_div

        @bempp.api.real_callable
        def d1_function(x, n, domain_index, result):
            result[:] = np.sum(atoms[:,3]/(impl1(x-atoms[:,:3])))

        d1_fun = bempp.api.GridFunction(neumann_stern, fun = d1_function) 
        if qtotal == 0:
            d1_mat = -(1/ep_iii) * d1_fun.coefficients
            #d1_mat = -(1/ep_iii) * d1_fun.coefficients/np.mean(d1_fun.coefficients)
            #d1_mat =   d1_mat- np.mean(d1_mat)
        else:
            d1_mat = -(qtotal/ep_iii) * d1_fun.coefficients/np.mean(d1_fun.coefficients)
        d1_gridfun = bempp.api.GridFunction(neumann_stern, coefficients = d1_mat)
        d1_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(d1_gridfun, 
                                                    neumann_stern, neumann_stern, neumann_stern)

        d2 = 1 

        sigma_1 = bempp.api.GridFunction(dirichlet_ion_1, coefficients = np.zeros(
            dirichlet_ion_1.global_dof_count))
        sigma_2 = bempp.api.GridFunction(dirichlet_ion_2, coefficients = np.zeros(
            dirichlet_ion_2.global_dof_count))

        for i in range(ITER): #Ec. 43
            blocked = bempp.api.BlockedOperator(6, 6)
            if i != 0:
                d2 = solution_neumann_stern.integrate()[0] #Ec. 42
                f_it_1 = f(sigma_1,en_1,K_L_I_a_a_adjoint,ep_i,ep_iii)
                f_fun_1 = bempp.api.GridFunction(neumann_ion_1, coefficients = f_it_1)
                f_op_1 = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun_1, 
                                        neumann_ion_1, neumann_ion_1, neumann_ion_1)
                f_it_2 = f(sigma_2,en_2,K_L_II_b_b_adjoint,ep_ii,ep_iii)
                f_fun_2 = bempp.api.GridFunction(neumann_ion_2, coefficients = f_it_2)
                f_op_2 = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun_2, 
                                        neumann_ion_2, neumann_ion_2, neumann_ion_2)
                blocked[1,1] = G_L_III_a_a * f_op_1 
                blocked[1,3] =  G_L_III_a_b * f_op_2 
                blocked[3,1] =   G_L_III_b_a * f_op_1 
                blocked[3,3] = G_L_III_b_b * f_op_2
                blocked[4,1] = G_L_III_c_a * f_op_1
                blocked[4,3] = G_L_III_c_b * f_op_2
                blocked[5,5] = (1/d2) * G_Y_IV_c_c * d1_op
            else:
                blocked[1,1] = G_L_III_a_a
                blocked[1,3] = G_L_III_a_b
                blocked[3,1] = G_L_III_b_a
                blocked[3,3] = G_L_III_b_b
                blocked[4,1] = G_L_III_c_a
                blocked[4,3] = G_L_III_c_b
                blocked[5,5] = (1/d2) * G_Y_IV_c_c

            blocked[0,0] = 0.5 * identity_ion_1 + K_L_I_a_a 
            blocked[0,1] = - G_L_I_a_a

            blocked[1,0] = 0.5 * identity_ion_1 - K_L_III_a_a
            blocked[1,2] = - K_L_III_a_b
            blocked[1,4] = K_L_III_a_c 
            blocked[1,5] = - G_L_III_a_c

            blocked[2,2] = 0.5 * identity_ion_2 + K_L_II_b_b
            blocked[2,3] = - G_L_II_b_b

            blocked[3,0] = - K_L_III_b_a
            blocked[3,2] = 0.5 * identity_ion_2 - K_L_III_b_b
            blocked[3,4] = K_L_III_b_c 
            blocked[3,5] = - G_L_III_b_c 

            blocked[4,0] = - K_L_III_c_a
            blocked[4,2] = - K_L_III_c_b 
            blocked[4,4] = 0.5 * identity_stern + K_L_III_c_c
            blocked[4,5] = - G_L_III_c_c

            blocked[5,4] = 0.5 * identity_stern - K_Y_IV_c_c

            blocked = blocked.strong_form()

            priiter(piter,i,en_sol)

            x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol=TOL)

            solution_dirichlet_ion_1 = bempp.api.GridFunction(dirichlet_ion_1, coefficients = 
                                                        x[:dirichlet_ion_1.global_dof_count])

            solution_neumann_ion_1 = bempp.api.GridFunction(neumann_ion_1, coefficients = 
                                                            x[dirichlet_ion_1.global_dof_count:
                                                             (dirichlet_ion_1.global_dof_count+
                                                              neumann_ion_1.global_dof_count)])

            solution_dirichlet_ion_2 = bempp.api.GridFunction(dirichlet_ion_2, coefficients = 
                                                                x[(dirichlet_ion_1.global_dof_count+
                                                                 neumann_ion_1.global_dof_count):
                                                                  (dirichlet_ion_1.global_dof_count+
                                                                 neumann_ion_1.global_dof_count+
                                                                  dirichlet_ion_2.global_dof_count)])

            solution_neumann_ion_2 = bempp.api.GridFunction(neumann_ion_2, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_ion_2.global_dof_count):
                                                             (dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count)])

            solution_dirichlet_stern = bempp.api.GridFunction(dirichlet_stern, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count):
                                                             (dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count+
                                                             dirichlet_stern.global_dof_count)])

            solution_neumann_stern = bempp.api.GridFunction(neumann_stern, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count+
                                                             dirichlet_stern.global_dof_count):])

            rhs_sigma_1 = G_L_I_a_a * solution_neumann_ion_1 - (- 0.5 * identity_ion_1 + K_L_I_a_a) * solution_dirichlet_ion_1

            rhs_sigma_2 = G_L_II_b_b * solution_neumann_ion_2 - (- 0.5 * identity_ion_2 + K_L_II_b_b) * solution_dirichlet_ion_2

            sigma_1, info_2 = bempp.api.linalg.gmres(G_L_I_a_a, rhs_sigma_1, maxiter=MAXITER_GM, tol=TOL)

            sigma_2, info_3 = bempp.api.linalg.gmres(G_L_II_b_b, rhs_sigma_2, maxiter=MAXITER_GM,tol=TOL)

            S0_q_1 = bempp.api.operators.potential.laplace.single_layer(neumann_ion_1, atoms_1[:,:3].transpose())
            D0_q_1 = bempp.api.operators.potential.laplace.double_layer(dirichlet_ion_1, atoms_1[:,:3].transpose())
            phi_q_1 = S0_q_1 * solution_neumann_ion_1 - D0_q_1 * solution_dirichlet_ion_1
            en_sol_1 = 2 * np.pi * 332.064 * np.sum(atoms_1[:,3] * phi_q_1).real

            S0_q_2 = bempp.api.operators.potential.laplace.single_layer(neumann_ion_2, atoms_2[:,:3].transpose())
            D0_q_2 = bempp.api.operators.potential.laplace.double_layer(dirichlet_ion_2, atoms_2[:,:3].transpose())
            phi_q_2 = S0_q_2 * solution_neumann_ion_2 - D0_q_2 * solution_dirichlet_ion_2
            en_sol_2 = 2 * np.pi * 332.064 * np.sum(atoms_2[:,3] * phi_q_2).real

            en_sol = en_sol_1 + en_sol_2

        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_dielectric_1_SLIC_PROP",solution_dirichlet_ion_1,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_dielectric_1_SLIC_PROP",solution_neumann_ion_1,atoms)

        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_dielectric_2_SLIC_PROP",solution_dirichlet_ion_2,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_dielectric_2_SLIC_PROP",solution_neumann_ion_2,atoms)
        
        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_stern_SLIC_PROP",solution_dirichlet_stern,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_stern_SLIC_PROP",solution_neumann_stern,atoms)

        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_dielectric_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', np.concatenate((solution_dirichlet_ion_1.coefficients,solution_dirichlet_ion_2.coefficients)), delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_dielectric_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', np.concatenate((solution_neumann_ion_1.coefficients,solution_neumann_ion_2.coefficients)), delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_stern_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', solution_dirichlet_stern.coefficients, delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_stern_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', solution_neumann_stern.coefficients, delimiter=",")
        
        
        
        info[key][index] = en_sol    
        privalues(pvalues,en_sol,key,atoms,"SLIC Proposed")
        energiesonly.write(str(en_sol)+',')
        
        phidphionly.write(',SLICPROP_phidielectric_1,'+str(np.mean(solution_dirichlet_ion_1.coefficients))+
                        ',SLICPROP_dphidielectric_1,'+str(np.mean(solution_neumann_ion_1.coefficients))+
                        ',SLICPROP_phidielectric_2,'+str(np.mean(solution_dirichlet_ion_2.coefficients))+
                        ',SLICPROP_dphidielectric_2,'+str(np.mean(solution_neumann_ion_2.coefficients))+
                        ',SLICPROP_phistern,'+str(np.mean(solution_dirichlet_stern.coefficients))+
                        ',SLICPROP_dphistern'+str(np.mean(solution_neumann_stern.coefficients)))

In [49]:
def separate_separate():
    en_sol = 0
    energiesonly.write('\n'+str(key)+',')
    phidphionly.write('\n'+str(key)+',')
    pristatus(pstatus,key)
    #Importan mallas de ion 
    grid_dielectric_1 = import_msms_mesh(MESHPATH+"\\"+key+"\\dielectric_1.face",
                                         MESHPATH+"\\"+key+"\\dielectric_1.vert")
    grid_dielectric_2 = import_msms_mesh(MESHPATH+"\\"+key+"\\dielectric_2.face",
                                         MESHPATH+"\\"+key+"\\dielectric_2.vert")

    grid_stern_1 = import_msms_mesh(MESHPATH+"\\"+key+"\\stern_1.face",
                                  MESHPATH+"\\"+key+"\\stern_1.vert")
    grid_stern_2 = import_msms_mesh(MESHPATH+"\\"+key+"\\stern_2.face",
                                  MESHPATH+"\\"+key+"\\stern_2.vert")

    #Se guarda malla en lista
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\dielectric_1",grid_dielectric_1)
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\dielectric_2",grid_dielectric_2)
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\stern_1",grid_stern_1)
    plot_mesh_to_file(MESHPATH+"\\"+key+"\\stern_2",grid_stern_2)

    #Ion 1
    dirichlet_ion_1 = bempp.api.function_space(grid_dielectric_1, "DP", 0)
    neumann_ion_1 = bempp.api.function_space(grid_dielectric_1, "DP", 0)

    #Stern 1
    dirichlet_stern_1 = bempp.api.function_space(grid_stern_1, "DP", 0)
    neumann_stern_1 = bempp.api.function_space(grid_stern_1, "DP", 0)

    #Ion 2
    dirichlet_ion_2 = bempp.api.function_space(grid_dielectric_2, "DP", 0)
    neumann_ion_2 = bempp.api.function_space(grid_dielectric_2, "DP", 0)

    #Stern 2
    dirichlet_stern_2 = bempp.api.function_space(grid_stern_2, "DP", 0)
    neumann_stern_2 = bempp.api.function_space(grid_stern_2, "DP", 0)

    om = 0 #Debye lenght
    ep_v = 80
    ep_iv = 80
    ep_iii = 1
    ep_ii = 80
    ep_i = 1

    #Operators:
    # Fila 1:
    identity_ion_1 = bempp.api.operators.boundary.sparse.identity(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)

    K_L_I_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)
    
    K_L_I_a_a_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)

    G_L_I_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_1, dirichlet_ion_1, neumann_ion_1)

    #Fila 2:
    K_L_II_a_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_1, dirichlet_ion_1, neumann_ion_1)

    G_L_II_a_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_1, dirichlet_ion_1, neumann_ion_1)

    K_L_II_a_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern_1, dirichlet_ion_1, neumann_ion_1)

    G_L_II_a_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern_1, dirichlet_ion_1, neumann_ion_1) 

    #Fila 3:
    K_L_II_b_a = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_1, dirichlet_stern_1, neumann_stern_1)

    G_L_II_b_a = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_1, dirichlet_stern_1, neumann_stern_1)

    identity_stern_1 = bempp.api.operators.boundary.sparse.identity(
        dirichlet_stern_1, dirichlet_stern_1, neumann_stern_1)

    K_L_II_b_b = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern_1, dirichlet_stern_1, neumann_stern_1)

    G_L_II_b_b = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern_1, dirichlet_stern_1, neumann_stern_1)

    #Fila 4:
    K_Y_V_b_b = bempp.api.operators.boundary.modified_helmholtz.double_layer(
        dirichlet_stern_1, dirichlet_stern_1, neumann_stern_1, om)

    G_Y_V_b_b = bempp.api.operators.boundary.modified_helmholtz.single_layer(
        neumann_stern_1, dirichlet_stern_1, neumann_stern_1, om)

    K_Y_V_b_d = bempp.api.operators.boundary.modified_helmholtz.double_layer(
        dirichlet_stern_2, dirichlet_stern_1, neumann_stern_1, om)

    G_Y_V_b_d = bempp.api.operators.boundary.modified_helmholtz.single_layer(
        neumann_stern_2, dirichlet_stern_1, neumann_stern_1, om)

    #Fila 5:
    identity_ion_2 = bempp.api.operators.boundary.sparse.identity(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)

    K_L_III_c_c = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)
    
    K_L_III_c_c_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)

    G_L_III_c_c = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_2, dirichlet_ion_2, neumann_ion_2)

    #Fila 6:
    K_L_IV_c_c = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_2, dirichlet_ion_2, neumann_ion_2)

    G_L_IV_c_c = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_2, dirichlet_ion_2, neumann_ion_2)

    K_L_IV_c_d = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern_2, dirichlet_ion_2, neumann_ion_2)

    G_L_IV_c_d = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern_2, dirichlet_ion_2, neumann_ion_2) 

    #Fila 7:
    K_L_IV_d_c = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_ion_2, dirichlet_stern_2, neumann_stern_2)

    G_L_IV_d_c = bempp.api.operators.boundary.laplace.single_layer(
        neumann_ion_2, dirichlet_stern_2, neumann_stern_2)

    identity_stern_2 = bempp.api.operators.boundary.sparse.identity(
        dirichlet_stern_2, dirichlet_stern_2, neumann_stern_2)

    K_L_IV_d_d = bempp.api.operators.boundary.laplace.double_layer(
        dirichlet_stern_2, dirichlet_stern_2, neumann_stern_2)

    G_L_IV_d_d = bempp.api.operators.boundary.laplace.single_layer(
        neumann_stern_2, dirichlet_stern_2, neumann_stern_2)

    #Fila 8:
    K_Y_V_d_b = bempp.api.operators.boundary.modified_helmholtz.double_layer(
        dirichlet_stern_1, dirichlet_stern_2, neumann_stern_2, om)

    G_Y_V_d_b = bempp.api.operators.boundary.modified_helmholtz.single_layer(
        neumann_stern_1, dirichlet_stern_2, neumann_stern_2, om)

    K_Y_V_d_d = bempp.api.operators.boundary.modified_helmholtz.double_layer(
        dirichlet_stern_2, dirichlet_stern_2, neumann_stern_2, om)

    G_Y_V_d_d = bempp.api.operators.boundary.modified_helmholtz.single_layer(
        neumann_stern_2, dirichlet_stern_2, neumann_stern_2, om)

    #SLIC proposed method
    for q1,q2,index in [(1,1,6),(1,-1,12),(-1,-1,16)]:
        atoms = np.array([[0,0,0,q1],[float(key),0,0,q2]])
        qtotal=np.sum(atoms[:,3])
        atoms_1 = np.array([atoms[0]])
        atoms_2 = np.array([atoms[1]])
        
        @bempp.api.real_callable
        def rhs_function_1(x, n, domain_index, result):

            result[:] = atoms_1[:,3]/(4 * np.pi * ep_i * impl1(x-atoms_1[:,:3]))

        @bempp.api.real_callable
        def rhs_function_2(x, n, domain_index, result):

            result[:] = atoms_2[:,3]/(4 * np.pi * ep_iii * impl1(x-atoms_2[:,:3]))

        # Se crea una GridFunction conteniendo el lado derecho de la equación:
        rhs_1 = bempp.api.GridFunction(dirichlet_ion_1, fun=rhs_function_1)
        rhs_2 = np.zeros(neumann_ion_1.global_dof_count)
        rhs_3 = np.zeros(dirichlet_stern_1.global_dof_count)
        rhs_4 = np.zeros(neumann_stern_1.global_dof_count)
        rhs_5 = bempp.api.GridFunction(dirichlet_ion_2, fun=rhs_function_2)
        rhs_6 = np.zeros(neumann_ion_2.global_dof_count)
        rhs_7 = np.zeros(dirichlet_stern_2.global_dof_count)
        rhs_8 = np.zeros(neumann_stern_2.global_dof_count)
        rhs = np.concatenate([rhs_1.coefficients,rhs_2,rhs_3,rhs_4,rhs_5.coefficients,rhs_6,rhs_7,rhs_8])

        
        @bempp.api.real_callable
        def en_function_1(x, n, domain_index, result):

            result[:] = np.sum(np.dot(n,(np.transpose(x-atoms_1[:,:3])/
                                         (4 * np.pi * impl1(x-atoms_1[:,:3])**3))*
                                      (atoms_1[:,3])))

        @bempp.api.real_callable
        def en_function_2(x, n, domain_index, result):

            result[:] = np.sum(np.dot(n,(np.transpose(x-atoms_2[:,:3])/
                                         (4 * np.pi * impl1(x-atoms_2[:,:3])**3))*
                                      (atoms_2[:,3])))
        en_1 = bempp.api.GridFunction(dirichlet_ion_1, fun=en_function_1)
        en_2 = bempp.api.GridFunction(dirichlet_ion_2, fun=en_function_2)
        
        def f(sigma,fun,K,ep_in,ep_out):
            Ksigma = K * sigma 
            En = fun - Ksigma
            mu = - alpha * np.tanh(- gamma)
            h = alpha * np.tanh(beta * En.coefficients - gamma) + mu
            f = ep_in/(ep_out - ep_in) - h
            f_div = f/(1+f)
            return f_div 

        d2 = 1
        d4 = 1
        # Se necesita el valor de la carga total de la proteína:
        qtotal_1=np.sum(atoms_1[:,3]) 
        qtotal_2=np.sum(atoms_2[:,3]) 

        @bempp.api.real_callable
        def d1_function(x, n, domain_index, result):
            result[:] = np.sum(atoms_1[:,3]/(impl1(x-atoms_1[:,:3])))

        d1_fun = bempp.api.GridFunction(neumann_stern_1, fun = d1_function) 
        d1_mat = -(qtotal_1/ep_ii) * d1_fun.coefficients/np.mean(d1_fun.coefficients)
        d1_gridfun = bempp.api.GridFunction(neumann_stern_1, coefficients = d1_mat)
        d1_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(d1_gridfun, 
                                                    neumann_stern_1, neumann_stern_1, neumann_stern_1)

        @bempp.api.real_callable
        def d3_function(x, n, domain_index, result):
            result[:] = np.sum(atoms_2[:,3]/(impl1(x-atoms_2[:,:3])))

        d3_fun = bempp.api.GridFunction(neumann_stern_2, fun = d3_function) 
        d3_mat = -(qtotal_2/ep_iv) * d3_fun.coefficients/np.mean(d3_fun.coefficients)
        d3_gridfun = bempp.api.GridFunction(neumann_stern_2, coefficients = d3_mat)
        d3_op = bempp.api.assembly.boundary_operator.MultiplicationOperator(d3_gridfun, 
                                                    neumann_stern_2, neumann_stern_2, neumann_stern_2)

        sigma_1 = bempp.api.GridFunction(dirichlet_ion_1, coefficients = np.zeros(
            dirichlet_ion_1.global_dof_count))
        sigma_2 = bempp.api.GridFunction(dirichlet_ion_2, coefficients = np.zeros(
            dirichlet_ion_2.global_dof_count))

        for i in range(ITER): #Ec. 43
            blocked = bempp.api.BlockedOperator(8, 8)
            if i != 0:
                d2 = solution_neumann_stern_1.integrate()[0] 
                d4 = solution_neumann_stern_2.integrate()[0] #Ec. 42
                f_it_1 = f(sigma_1,en_1,K_L_I_a_a_adjoint,ep_i,ep_ii)
                f_fun_1 = bempp.api.GridFunction(neumann_ion_1, coefficients = f_it_1)
                f_op_1 = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun_1, 
                                        neumann_ion_1, neumann_ion_1, neumann_ion_1)
                f_it_2 = f(sigma_2,en_2,K_L_III_c_c_adjoint,ep_iii,ep_iv)
                f_fun_2 = bempp.api.GridFunction(neumann_ion_2, coefficients = f_it_2)
                f_op_2 = bempp.api.assembly.boundary_operator.MultiplicationOperator(f_fun_2, 
                                        neumann_ion_2, neumann_ion_2, neumann_ion_2)
                blocked[1,1] =  G_L_II_a_a * f_op_1
                blocked[2,1] = G_L_II_b_a * f_op_1
                blocked[3,3] = (1/d2) * G_Y_V_b_b * d1_op
                blocked[3,7] = (1/d4) * G_Y_V_b_d * d3_op
                blocked[5,5] = G_L_IV_c_c * f_op_2
                blocked[6,5] = G_L_IV_d_c * f_op_2
                blocked[7,3] = (1/d2) * G_Y_V_d_b * d1_op 
                blocked[7,7] = (1/d4) * G_Y_V_d_d * d3_op

            else:
                blocked[1,1] = G_L_II_a_a
                blocked[2,1] = G_L_II_b_a
                blocked[3,3] = G_Y_V_b_b
                blocked[3,7] = G_Y_V_b_d
                blocked[5,5] = G_L_IV_c_c
                blocked[6,5] = G_L_IV_d_c
                blocked[7,3] = G_Y_V_d_b  
                blocked[7,7] = G_Y_V_d_d


            blocked[0,0] = 0.5 * identity_ion_1 + K_L_I_a_a 
            blocked[0,1] = - G_L_I_a_a

            blocked[1,0] = 0.5 * identity_ion_1 - K_L_II_a_a
            blocked[1,2] =  K_L_II_a_b
            blocked[1,3] = - G_L_II_a_b

            blocked[2,0] = - K_L_II_b_a 
            blocked[2,2] = 0.5 * identity_stern_1 + K_L_II_b_b
            blocked[2,3] = - G_L_II_b_b

            blocked[3,2] = 0.5 * identity_stern_1 - K_Y_V_b_b
            blocked[3,6] = - K_Y_V_b_d

            blocked[4,4] = 0.5 * identity_ion_2 + K_L_III_c_c 
            blocked[4,5] = - G_L_III_c_c

            blocked[5,4] = 0.5 * identity_ion_2- K_L_IV_c_c
            blocked[5,6] =  K_L_IV_c_d
            blocked[5,7] = - G_L_IV_c_d

            blocked[6,4] = - K_L_IV_d_c 
            blocked[6,6] = 0.5 * identity_stern_2 + K_L_IV_d_d
            blocked[6,7] = - G_L_IV_d_d

            blocked[7,2] = - K_Y_V_d_b  
            blocked[7,6] = 0.5 * identity_stern_2 - K_Y_V_d_d

            blocked = blocked.strong_form()

            priiter(piter,i,en_sol)

            x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol = TOL)

            solution_dirichlet_ion_1 = bempp.api.GridFunction(dirichlet_ion_1, coefficients = 
                                                        x[:dirichlet_ion_1.global_dof_count])

            solution_neumann_ion_1 = bempp.api.GridFunction(neumann_ion_1, coefficients = 
                                                            x[dirichlet_ion_1.global_dof_count:
                                                             (dirichlet_ion_1.global_dof_count+
                                                              neumann_ion_1.global_dof_count)])

            solution_dirichlet_stern_1 = bempp.api.GridFunction(dirichlet_stern_1, coefficients = 
                                                                x[(dirichlet_ion_1.global_dof_count+
                                                                 neumann_ion_1.global_dof_count):
                                                                  (dirichlet_ion_1.global_dof_count+
                                                                 neumann_ion_1.global_dof_count+
                                                                  dirichlet_stern_1.global_dof_count)])

            solution_neumann_stern_1 = bempp.api.GridFunction(neumann_stern_1, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count):
                                                             (dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count)])

            solution_dirichlet_ion_2 = bempp.api.GridFunction(dirichlet_ion_2, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count):
                                                             (dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count+
                                                             dirichlet_ion_2.global_dof_count)])

            solution_neumann_ion_2 = bempp.api.GridFunction(neumann_ion_2, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count+
                                                             dirichlet_ion_2.global_dof_count):
                                                             (dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count+
                                                             dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count)])

            solution_dirichlet_stern_2 = bempp.api.GridFunction(dirichlet_stern_2, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count+
                                                             dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count):
                                                              (dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count+
                                                             dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count+
                                                              dirichlet_stern_2.global_dof_count)])

            solution_neumann_stern_2 = bempp.api.GridFunction(neumann_stern_2, coefficients = 
                                                            x[(dirichlet_ion_1.global_dof_count+
                                                             neumann_ion_1.global_dof_count+
                                                              dirichlet_stern_1.global_dof_count+
                                                             neumann_stern_1.global_dof_count+
                                                             dirichlet_ion_2.global_dof_count+
                                                             neumann_ion_2.global_dof_count+
                                                              dirichlet_stern_2.global_dof_count):])

            rhs_sigma_1 = G_L_I_a_a * solution_neumann_ion_1 - (- 0.5 * identity_ion_1 + K_L_I_a_a) * solution_dirichlet_ion_1

            rhs_sigma_2 = G_L_III_c_c * solution_neumann_ion_2 - (- 0.5 * identity_ion_2 + K_L_III_c_c) * solution_dirichlet_ion_2

            sigma_1, info_2 = bempp.api.linalg.gmres(G_L_I_a_a, rhs_sigma_1, maxiter=MAXITER_GM ,tol=TOL)

            sigma_2, info_3 = bempp.api.linalg.gmres(G_L_III_c_c, rhs_sigma_2, tol=TOL, maxiter = MAXITER_GM)

            S0_q_1 = bempp.api.operators.potential.laplace.single_layer(neumann_ion_1, atoms_1[:,:3].transpose())
            D0_q_1 = bempp.api.operators.potential.laplace.double_layer(dirichlet_ion_1, atoms_1[:,:3].transpose())
            phi_q_1 = S0_q_1 * solution_neumann_ion_1 - D0_q_1 * solution_dirichlet_ion_1
            en_sol_1 = 2 * np.pi * 332.064 * np.sum(atoms_1[:,3] * phi_q_1).real

            S0_q_2 = bempp.api.operators.potential.laplace.single_layer(neumann_ion_2, atoms_2[:,:3].transpose())
            D0_q_2 = bempp.api.operators.potential.laplace.double_layer(dirichlet_ion_2, atoms_2[:,:3].transpose())
            phi_q_2 = S0_q_2 * solution_neumann_ion_2 - D0_q_2 * solution_dirichlet_ion_2
            en_sol_2 = 2 * np.pi * 332.064 * np.sum(atoms_2[:,3] * phi_q_2).real

            en_sol = en_sol_1 + en_sol_2
            
        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_dielectric_1_SLIC_PROP",solution_dirichlet_ion_1,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_dielectric_1_SLIC_PROP",solution_neumann_ion_1,atoms)

        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_dielectric_2_SLIC_PROP",solution_dirichlet_ion_2,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_dielectric_2_SLIC_PROP",solution_neumann_ion_2,atoms)

        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_stern_SLIC_PROP",solution_dirichlet_stern_1,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_stern_SLIC_PROP",solution_neumann_stern_1,atoms)

        plot_solution_to_file(MESHPATH+"\\"+key+"\\dirichlet_stern_SLIC_PROP",solution_dirichlet_stern_2,atoms)
        plot_solution_to_file(MESHPATH+"\\"+key+"\\neumann_stern_SLIC_PROP",solution_neumann_stern_2,atoms)

        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_dielectric_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', np.concatenate((solution_dirichlet_ion_1.coefficients,solution_dirichlet_ion_2.coefficients)), delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_dielectric_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', np.concatenate((solution_neumann_ion_1.coefficients,solution_neumann_ion_2.coefficients)), delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_dirichlet_stern_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', np.concatenate((solution_dirichlet_stern_1.coefficients,solution_dirichlet_stern_2.coefficients)), delimiter=",")
        np.savetxt(MESHPATH+'\\'+key+'\\SLICPROP_neumann_stern_'+str(atoms[0][3])+'_'+str(atoms[1][3])+'.csv', np.concatenate((solution_neumann_stern_1.coefficients,solution_neumann_stern_2.coefficients)), delimiter=",")
        
            
        
        info[key][index] = en_sol
        privalues(pvalues,en_sol,key,atoms,"SLIC Proposed")
        energiesonly.write(str(en_sol)+',')
        phidphionly.write(',SLICPROP_phidielectric_1,'+str(np.mean(solution_dirichlet_ion_1.coefficients))+
                        ',SLICPROP_dphidielectric_1,'+str(np.mean(solution_neumann_ion_1.coefficients))+
                        ',SLICPROP_phidielectric_2,'+str(np.mean(solution_dirichlet_ion_2.coefficients))+
                        ',SLICPROP_dphidielectric_2,'+str(np.mean(solution_neumann_ion_2.coefficients))+
                        ',SLICPROP_phistern_1,'+str(np.mean(solution_dirichlet_stern_1.coefficients))+
                        ',SLICPROP_dphistern_1,'+str(np.mean(solution_neumann_stern_1.coefficients))+
                        ',SLICPROP_phistern_2,'+str(np.mean(solution_dirichlet_stern_2.coefficients))+
                        ',SLICPROP_dphistern_2'+str(np.mean(solution_neumann_stern_2.coefficients)))

In [50]:
os.chdir(basedir)
en_sol = 0
energiesonly = open('results.txt','w')
phidphionly = open('results_phidphi.txt','w')
for key in info.keys():
    if key == "ion":
        ion()
        
    if  key!="ion" and info[key][0] == "shared" and info[key][1] == "shared":
        shared_shared()
        
    if  info[key][0] == "separate" and info[key][1] == "shared":
        separate_shared()
        
    if  info[key][0] == "separate" and info[key][1] == "separate":
        separate_separate()
energiesonly.close()
phidphionly.close()

Solving for ion


D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



Iteration: 0, Energía: 0
Iteration: 1, Energía: -0.010735501108404374
Iteration: 2, Energía: -94.7657910648078
Iteration: 3, Energía: -102.76156794063817
Iteration: 4, Energía: -103.54417955826969
Iteration: 5, Energía: -103.6165428652094
Iteration: 6, Energía: -103.62303116491633
Iteration: 7, Energía: -103.62360613182628
Iteration: 8, Energía: -103.62365687091376
Iteration: 9, Energía: -103.62366134195892
Iteration: 10, Energía: -103.62366173572492
Iteration: 11, Energía: -103.62366177038739
Iteration: 12, Energía: -103.62366177347396
Iteration: 13, Energía: -103.6236617737214
Iteration: 14, Energía: -103.6236617737448
Iteration: 15, Energía: -103.62366177375993
Iteration: 16, Energía: -103.62366177375338
Iteration: 17, Energía: -103.62366177376981
Iteration: 18, Energía: -103.62366177376379
Iteration: 19, Energía: -103.62366177376953
Solvation Energy, ion, SLIC Proposed: -103.62366177376312
[[0 0 0 1]]
Iteration: 0, Energía: -103.62366177376312
Iteration: 1, Energía: -0.010735501108

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



Iteration: 0, Energía: 0
Iteration: 1, Energía: -0.07193371055816614
Iteration: 2, Energía: -279.7861731813326
Iteration: 3, Energía: -299.6916882867391
Iteration: 4, Energía: -301.1666794342743
Iteration: 5, Energía: -301.25301045892627
Iteration: 6, Energía: -301.2568102013357
Iteration: 7, Energía: -301.2569028570216
Iteration: 8, Energía: -301.25689931489427
Iteration: 9, Energía: -301.25689863885754
Iteration: 10, Energía: -301.2568985824356
Iteration: 11, Energía: -301.25689857904325
Iteration: 12, Energía: -301.256898578895
Iteration: 13, Energía: -301.25689857889193
Iteration: 14, Energía: -301.2568985788921
Iteration: 15, Energía: -301.2568985788921
Iteration: 16, Energía: -301.25689857889205
Iteration: 17, Energía: -301.25689857889205
Iteration: 18, Energía: -301.256898578892
Iteration: 19, Energía: -301.256898578892
Solvation Energy, 3.0, SLIC Proposed: -301.25689857889205
[[0. 0. 0. 1.]
 [3. 0. 0. 1.]]
Iteration: 0, Energía: -301.25689857889205
Iteration: 1, Energía: -0.164

...

Iteration: 0, Energía: 0
Iteration: 1, Energía: 36.87325731229743
Iteration: 2, Energía: -187.02140774086104
Iteration: 3, Energía: -204.91164451490602
Iteration: 4, Energía: -206.56438677592968
Iteration: 5, Energía: -206.6996675210074
Iteration: 6, Energía: -206.70976960594325
Iteration: 7, Energía: -206.71047769012927
Iteration: 8, Energía: -206.7105250820058
Iteration: 9, Energía: -206.71052806685213
Iteration: 10, Energía: -206.7105283129122
Iteration: 11, Energía: -206.71052826693528
Iteration: 12, Energía: -206.7105283248995
Iteration: 13, Energía: -206.71052826764662
Iteration: 14, Energía: -206.71052832493638
Iteration: 15, Energía: -206.71052826765742
Iteration: 16, Energía: -206.7105283249067
Iteration: 17, Energía: -206.71052826763076
Iteration: 18, Energía: -206.71052832488562
Iteration: 19, Energía: -206.71052826764895
Solvation Energy, 9.0, SLIC Proposed: -206.7105283248981
[[0. 0. 0. 1.]
 [9. 0. 0. 1.]]
Iteration: 0, Energía: -206.7105283248981
Iteration: 1, Energía: -3

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



Iteration: 0, Energía: 0
Iteration: 1, Energía: 34.932392146334436
Iteration: 2, Energía: -187.14844129962657
Iteration: 3, Energía: -204.94463953831598
Iteration: 4, Energía: -206.59416844062343
Iteration: 5, Energía: -206.73023648923052
Iteration: 6, Energía: -206.74053066666886
Iteration: 7, Energía: -206.7412660872713
Iteration: 8, Energía: -206.74131652198463
Iteration: 9, Energía: -206.74131987330634
Iteration: 10, Energía: -206.74132009027326
Iteration: 11, Energía: -206.74132010400314
Iteration: 12, Energía: -206.74132010485673
Iteration: 13, Energía: -206.7413201049127
Iteration: 14, Energía: -206.7413201049181
Iteration: 15, Energía: -206.74132010494745
Iteration: 16, Energía: -206.74132010494543
Iteration: 17, Energía: -206.7413201049526
Iteration: 18, Energía: -206.7413201049116
Iteration: 19, Energía: -206.74132010492684
Solvation Energy, 9.5, SLIC Proposed: -206.74132010495947
[[0.  0.  0.  1. ]
 [9.5 0.  0.  1. ]]
Iteration: 0, Energía: -206.74132010495947
Iteration: 1, 

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning:

splu requires CSC matrix format



Iteration: 0, Energía: 0
Iteration: 1, Energía: 33.18539456210786
Iteration: 2, Energía: -187.26311070448344
Iteration: 3, Energía: -204.9740184648872
Iteration: 4, Energía: -206.62045535086347
Iteration: 5, Energía: -206.7571880223108
Iteration: 6, Energía: -206.76764881975336
Iteration: 7, Energía: -206.76840814894967
Iteration: 8, Energía: -206.7684613257589
Iteration: 9, Energía: -206.7684649538254
Iteration: 10, Energía: -206.76846519639687
Iteration: 11, Energía: -206.76846521239725
Iteration: 12, Energía: -206.76846521341182
Iteration: 13, Energía: -206.76846521346766
Iteration: 14, Energía: -206.76846521348568
Iteration: 15, Energía: -206.76846521350902
Iteration: 16, Energía: -206.76846521350242
Iteration: 17, Energía: -206.76846521350296
Iteration: 18, Energía: -206.7684652135052
Iteration: 19, Energía: -206.76846521350393
Solvation Energy, 10.0, SLIC Proposed: -206.76846521348426
[[ 0.  0.  0.  1.]
 [10.  0.  0.  1.]]
Iteration: 0, Energía: -206.76846521348426
Iteration: 1, 

In [10]:
grid_dielectric = import_msms_mesh(MESHPATH+"\\ion\\dielectric.face",
                                       MESHPATH+"\\ion\\dielectric.vert")

grid_stern = import_msms_mesh(MESHPATH+"\\ion\\stern.face",
                              MESHPATH+"\\ion\\stern.vert")

# Se crean los espacios funcionales:
#Protein
dirichlet_ion = bempp.api.function_space(grid_dielectric, "DP", 0)
neumann_ion = bempp.api.function_space(grid_dielectric, "DP", 0)

#Stern
dirichlet_stern = bempp.api.function_space(grid_stern, "DP", 0)
neumann_stern = bempp.api.function_space(grid_stern, "DP", 0)

om = 0 #Debye lenght
ep_i = 1
ep_ii = 80
ep_iii = 80

#Operators:
identity_ion = bempp.api.operators.boundary.sparse.identity(
    dirichlet_ion, dirichlet_ion, neumann_ion)

K_L_I_a_a = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_ion, dirichlet_ion, neumann_ion)

K_L_I_a_a_adjoint = bempp.api.operators.boundary.laplace.adjoint_double_layer(
    dirichlet_ion, dirichlet_ion, neumann_ion)

V_L_I_a_a = bempp.api.operators.boundary.laplace.single_layer(
    neumann_ion, dirichlet_ion, neumann_ion)

K_L_II_a_a = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_ion, dirichlet_ion, neumann_ion)

V_L_II_a_a = bempp.api.operators.boundary.laplace.single_layer(
    neumann_ion, dirichlet_ion, neumann_ion)

K_L_II_a_b = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_stern, dirichlet_ion, neumann_ion) #OJO ACA

V_L_II_a_b = bempp.api.operators.boundary.laplace.single_layer(
    neumann_stern, dirichlet_ion, neumann_ion) #OJO ACA

K_L_II_b_a = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_ion, dirichlet_stern, neumann_stern) #OJO ACA

V_L_II_b_a = bempp.api.operators.boundary.laplace.single_layer(
    neumann_ion, dirichlet_stern, neumann_stern) #OJO ACA

identity_stern = bempp.api.operators.boundary.sparse.identity(
    dirichlet_stern, dirichlet_stern, neumann_stern)

K_L_II_b_b = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_stern, dirichlet_stern, neumann_stern)

V_L_II_b_b = bempp.api.operators.boundary.laplace.single_layer(
    neumann_stern, dirichlet_stern, neumann_stern)

K_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.double_layer(
    dirichlet_stern, dirichlet_stern, neumann_stern, om)

V_Y_III_b_b = bempp.api.operators.boundary.modified_helmholtz.single_layer(
    neumann_stern, dirichlet_stern, neumann_stern, om)

atoms = np.array([[0,0,0,1]])

# Se mapea la función analítica del lado derecho de la equación a la malla creada en Bempp:
@bempp.api.real_callable
def rhs_function(x, n, domain_index, result):
    result[:] = np.sum(atoms[:,3]/(4 * np.pi * ep_i * impl1(x-atoms[:,:3])))

# Se crea una GridFunction conteniendo el lado derecho de la equación:
rhs_1 = bempp.api.GridFunction(dirichlet_ion, fun=rhs_function)
rhs_2 = np.zeros(neumann_ion.global_dof_count)
rhs_3 = np.zeros(dirichlet_stern.global_dof_count)
rhs_4 = np.zeros(neumann_stern.global_dof_count)
rhs = np.concatenate([rhs_1.coefficients,rhs_2,rhs_3,rhs_4])

#PB Calculation            
blocked = bempp.api.BlockedOperator(4, 4)

blocked[0,0] = 0.5 * identity_ion + K_L_I_a_a 
blocked[0,1] = - V_L_I_a_a

blocked[1,0] = 0.5 * identity_ion - K_L_II_a_a
blocked[1,1] = (ep_i/ep_ii) * V_L_II_a_a
blocked[1,2] = K_L_II_a_b 
blocked[1,3] = - V_L_II_a_b

blocked[2,0] = - K_L_II_b_a
blocked[2,1] = (ep_i/ep_ii) * V_L_II_b_a 
blocked[2,2] = 0.5 * identity_stern + K_L_II_b_b
blocked[2,3] = - V_L_II_b_b

blocked[3,2] = 0.5 * identity_stern - K_Y_III_b_b
blocked[3,3] =  (ep_ii/ep_iii) * V_Y_III_b_b  

blocked = blocked.strong_form()

x, info_1 = gmres(blocked, rhs, maxiter=MAXITER_GM, tol = TOL)

solution_neumann_stern = bempp.api.GridFunction(neumann_stern, coefficients = 
                                                x[(dirichlet_ion.global_dof_count+
                                                 neumann_ion.global_dof_count+
                                                  dirichlet_stern.global_dof_count):])

solution_neumann_ion = bempp.api.GridFunction(neumann_ion, coefficients = 
                                                x[dirichlet_ion.global_dof_count:
                                                 (dirichlet_ion.global_dof_count+
                                                  neumann_ion.global_dof_count)])

solution_dirichlet_ion = bempp.api.GridFunction(dirichlet_ion, coefficients = 
                                                x[:dirichlet_ion.global_dof_count])

solution_dirichlet_stern = bempp.api.GridFunction(dirichlet_stern, coefficients = 
                                                    x[(dirichlet_ion.global_dof_count+
                                                     neumann_ion.global_dof_count):
                                                      (dirichlet_ion.global_dof_count+
                                                     neumann_ion.global_dof_count+
                                                      dirichlet_stern.global_dof_count)])

S0_q = bempp.api.operators.potential.laplace.single_layer(neumann_ion, atoms[:,:3].transpose())
D0_q = bempp.api.operators.potential.laplace.double_layer(dirichlet_ion, atoms[:,:3].transpose())
phi_q = S0_q * solution_neumann_ion - D0_q * solution_dirichlet_ion
en_sol = 2 * np.pi * 332.064 * np.sum(atoms[:,3] * phi_q).real
print(en_sol)

D:\ProgramData\Anaconda3\envs\bemppenv\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


-130.01475964610503
